<a href="https://colab.research.google.com/github/Rohitha2803/CAPSTONE-PROJECT/blob/main/edututor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate streamlit pyngrok



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
import os

# Store token securely (for demo purposes here only)
os.environ["HUGGINGFACE_TOKEN"] = ""


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# Load model and tokenizer
model_name = "ibm-granite/granite-3.3-2b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.environ["HUGGINGFACE_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(model_name,
                                              use_auth_token=os.environ["HUGGINGFACE_TOKEN"],
                                              torch_dtype=torch.float16,
                                              device_map="auto")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: ibm-granite/granite-3.3-2b-instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from PyPDF2 import PdfReader

# Load model
@st.cache_resource
def load_model():
    model_name = "ibm-granite/granite-3.3-2b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        use_auth_token=True,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    return tokenizer, model

tokenizer, model = load_model()

# Function to extract text from PDF
def extract_text_from_pdf(uploaded_file):
    reader = PdfReader(uploaded_file)
    text = ""
    for page in reader.pages:
        if page.extract_text():
            text += page.extract_text()
    return text

# Function to generate MCQs using the model
def generate_mcqs(text):
    prompt = f"Generate 15 multiple choice questions (with 4 options each and correct answer key) based on the following text:\n{text[:3500]}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Parse MCQs from the model response
def parse_mcqs(raw_text):
    import re
    questions = []
    current = {}
    for line in raw_text.split("\n"):
        line = line.strip()
        if re.match(r"^\d+\.", line):
            if current:
                questions.append(current)
            current = {"question": line, "options": [], "answer": ""}
        elif re.match(r"^[a-dA-D]\)", line):
            current["options"].append(line)
        elif "Answer:" in line or "Correct Answer:" in line:
            current["answer"] = line.split(":")[-1].strip().lower()
    if current:
        questions.append(current)
    return questions[:15]  # Limit to 15

# Sidebar
st.sidebar.title("EDUtutor AI")
page = st.sidebar.radio("Choose an Option", ["MCQ Test from PDF", "LMS Assistant"])

# ---------- MCQ TEST ----------
if page == "MCQ Test from PDF":
    st.title("📄 Upload PDF to Generate MCQ Quiz")

    uploaded_file = st.file_uploader("Upload your educational PDF", type="pdf")

    if uploaded_file:
        with st.spinner("Reading PDF and generating questions..."):
            pdf_text = extract_text_from_pdf(uploaded_file)
            raw_mcqs = generate_mcqs(pdf_text)
            questions = parse_mcqs(raw_mcqs)

        if questions:
            st.success("✅ Questions generated. Answer below:")
            user_answers = []
            for idx, q in enumerate(questions):
                st.markdown(f"Q{idx+1}. {q['question']}")
                selected = st.radio(f"Choose answer:", q['options'], key=idx)
                if selected:
                    user_answers.append((selected.strip().lower()[0], q["answer"]))
                else:
                    user_answers.append(("", q["answer"]))

            if st.button("Submit Answers"):
                if any(ans[0] == "" for ans in user_answers):
                    st.warning("⚠ Please answer all questions before submitting.")
                    st.stop()
                score = sum([1 for user, correct in user_answers if user == correct])
                st.success(f"🎉 Your Score: {score} / {len(questions)}")
        else:
            st.warning("Couldn't generate valid questions. Try a different PDF or refine content.")

# ---------- LMS ASSISTANT ----------
elif page == "LMS Assistant":
    st.title("📚 Lms Integration (Coming Soon)")
    st.write("Integrate Moodle or Google Classroom to fetch and explain assignments, grades, and more.")

Overwriting app.py


In [ ]:


!ngrok config add-authtoken 2uDutWeemqWp280nWnzNsss3gUv_5eMFAG1EN3vwaazjJ1REH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/dev/null &


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit App is Live at: {public_url}")


🚀 Streamlit App is Live at: NgrokTunnel: "https://ab76-34-125-135-188.ngrok-free.app" -> "http://localhost:8501"
